<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">
     
# Parallelize your python code

In this lesson you will learn how to paralellize custom python code using Dask. You will learn about the Futures API, and how to use it to parallelize customs functions.

## Futures: a low-level collection.

Dask low-level collections are the best tools when you need to have fine control control to build custom parallel and distributed computations. 

The `futures` interface (derived from the built-in `concurrent.futures`) provides fine-grained real-time execution for custom situations. It allows you to submit arbitrary functions for computation in a parallelized, eager, and non-blocking way. 

### Why use Futures?

The `futures` API offers a work submission style that can easily emulate the map/reduce paradigm. If that is familiar to you then futures might be the simplest entrypoint into Dask.

The other big benefit of futures is that the intermediate results, represented by futures, can be passed to new tasks without having to pull data locally from the cluster. The **call returns immediately**, giving one or more *futures*, whose status begins as "pending" and later becomes "finished". There is no blocking of the local Python session. With futures, as soon as the inputs are available and there is compute available, the computation starts. 

One of the most common cases where you can use `Futures` is when you have a for loop. For example, you need to apply a read-transform-write function over multiple files, where you serial code will look something like


```python
def process_file(filename):
    data = read_a_file(filename)
    data_transformed = do_a_transformation(data)
    destination = f"results/{filename}"
    write_out_data(data_transformed, destination)
    return destination

files = ["file_1", "file_2", "file_3", ..., "file_n"] #list of files
new_files = [] #where we save the destination file names

for f in files:
    new_files.append(process_file(f)            
```

In the code above, every call of `process_file` is independent from each other, this is what is call an embarrassingly parallel problem. You can do this in parallel by doing

```python
futures = []
for f in files:
    future = client.submit(process_file, f)
    futures.append(future)
    
futures
```

## Example: Get SO get questions page title 

During this lesson, we will be working with the Stack Overflow question pages. To start let's see how to grab the title of each page, and how for multiple pages we can perform this in parallel. 

If you go to https://stackoverflow.com/questions/ you will see a list of the newest posts, if you to the bottom of the page you can switch to the next page. For example, the top of page number two at the moment of the creation of this notebook, looked like 

<center>
<img src="images/SO_page.png"
     width="65%"
     alt="SO page\">
</center>

### Get the title

The title of the page is what is showed in the tab. The following function gets the title of a page, given it's page number:

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time

In [22]:
def get_questions_page_title(page_num):
    """ Get title of a SO questions page
    """
    url = f"https://stackoverflow.com/questions?tab=newest&page={page_num}"
    req = requests.get(url)
    html = bs(req.text, "html.parser")

    return html.title.text

In [23]:
page_2 = get_questions_page_title(2)
page_2

### Serial code to get 8 pages would be:

In [6]:
%%time
page_html = []
for p in range(1,9): #page numbers start in 1
    page_html.append(get_questions_page_title(p))
    

CPU times: user 1e+03 ms, sys: 31.4 ms, total: 1.03 s
Wall time: 2.8 s


In [8]:
page_html[:3]

['Newest Questions - Page 1 - Stack Overflow',
 'Newest Questions - Page 2 - Stack Overflow',
 'Newest Questions - Page 3 - Stack Overflow']

### Exercise

Run the code in parallel, using futures.

In [3]:
from dask.distributed import Client, wait

In [4]:
client = Client(n_workers=4)
client

2022-12-28 10:46:39,970 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1y/ydztfpnd11b6qmvbb8_x56jh0000gn/T/dask-worker-space/worker-0k2dk3gl', purging
2022-12-28 10:46:39,972 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1y/ydztfpnd11b6qmvbb8_x56jh0000gn/T/dask-worker-space/worker-b44rsipe', purging
2022-12-28 10:46:39,973 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1y/ydztfpnd11b6qmvbb8_x56jh0000gn/T/dask-worker-space/worker-__n0x7p2', purging
2022-12-28 10:46:39,974 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/1y/ydztfpnd11b6qmvbb8_x56jh0000gn/T/dask-worker-space/worker-oikr1b7x', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61813,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:61825,Total threads: 2
Dashboard: http://127.0.0.1:61829/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:61819,


In [11]:
#Solution
futures = []
for p in range(1,9):
    future = client.submit(get_questions_page_title, p)
    futures.append(future)
    
futures

[<Future: pending, key: get_questions_page_title-cf6b290a43bbc50b6b7ae36d0bea825c>,
 <Future: pending, key: get_questions_page_title-ea19e73919b66b229f06298dd0296f66>,
 <Future: pending, key: get_questions_page_title-a7eee38cc4e44a007b4af84a4dbcdfda>,
 <Future: pending, key: get_questions_page_title-c5f800008e189c6188c490aeda41ca28>,
 <Future: pending, key: get_questions_page_title-84a443272ee63b7d105e90d2fc8c9d7c>,
 <Future: pending, key: get_questions_page_title-5b408944028ad056224128a8ee0a680f>,
 <Future: pending, key: get_questions_page_title-965a4ee509af33e806162b35830cb692>,
 <Future: pending, key: get_questions_page_title-1cc10177431e28480edf4972c7d585c6>]

In [12]:
futures[0]

<Future: finished, type: str, key: get_questions_page_title-cf6b290a43bbc50b6b7ae36d0bea825c>

In [13]:
futures[0].result()

'Newest Questions - Page 1 - Stack Overflow'

In [ ]:
results = [future.result() for future in futures]
results

**Extra:**

To be able to `%%time` the cell and compare times with the serial version, you will need to wait for the futures to finish doing `wait(futures)`. If you try to do that and re run the cell, you will notice it is immediate, this is because by default, distributed assumes that all functions are pure. Pure functions:

- always return the same output for a given set of inputs
- do not have side effects, like modifying global state or creating files

 
You can use the `pure=False` keyword argument in the `client.submit()`. Modify your solution to match this code


```python
%%time
futures = []
for p in range(1,9):
    future = client.submit(get_questions_page_title, p, pure=False)
    futures.append(future)
    
wait(futures)
```

## TODO
- replace code such that request we get html, 1 func to get html 1 func that takes it and gets links 
    and ass completed pass it to get post links

- transition to complicated example where we have an extra function to get data

- we also need motivation and show a plot or something. 
- something like (histogram with people usrname and +votes, +most answered questions)

```
page_2.find_all("a", class_="s-link")[3:-1][0]

t = page_2.find_all("a", class_="s-link")[3:-1][0]['href']

```

**`client.map()`**

With `client.submit()` you can submit individual functions for evaluation with one set of inputs, and together with a `for-loop` you can also evaluate over a sequence of inputs. `client.map()` provides a simpler interface to perform the former, let's see how to perform the example above now using `client.map()`


In [ ]:
futures = client.map(get_questions_page_title, range(1, 9))

`client.map()` returns a list of futures, you can block on the computation and gather the result by doing:

In [ ]:
res = client.gather(futures)
res

### Futures are great...

The other big benefit of futures is that the intermediate results, represented by futures, can be passed to new tasks without having to pull data locally from the cluster. New operations can be setup to work on the output of previous jobs that haven’t even begun yet.

Let's brake our steps into multiple functions

- `request_html_page`: given a url returns the html of that page
- `get_page_html_links`: given a SO questions page number returns a the html for that page number.
- `get_post_links_per_page`: given a SO questions html page, returns a list with the posts of that page.


##ADD IMAGE WITH TWO PAGES AND POINTING TO LINKS. USE SAME IMAGE CIRCLING LINKS

In [5]:
def request_html_page(url):
    """Given a url returns the html of that page
    """
    req = requests.get(url)
    html = bs(req.text, "html.parser")
    return html

In [6]:
def get_page_html_links(page_num, tag="dask", query_filter = "MostVotes"):
    """Given a SO questions page number returns a the html for that page number
    for a tag and query_filter.
    """
    base_url = "https://stackoverflow.com/questions/tagged/"
    
    page_url = f"{base_url}{tag}?sort={query_filter}&page={page_num}"

    page_html = request_html_page(page_url)
    
    return page_html

In [26]:
def get_post_links_per_page(html_page):
    """Given a SO questions html page, returns a list with the posts of that page.
    """
    question_href = html_page.find_all("a", class_="s-link")[2:-1]
    
    question_link = [f"https://stackoverflow.com{q['href']}" for q in question_href]
    
    return question_link

### Explore the functions:  

In [33]:
page_number = 3

page_3_html = get_page_html_links(page_num=page_number)

In [34]:
post_links_page_3 = get_post_links_per_page(page_3_html)

In [35]:
len(post_links_page_3)

50

In [36]:
post_links_page_3[:3]

['https://stackoverflow.com/questions/58795861/only-a-column-name-can-be-used-for-the-key-in-a-dtype-mappings-argument',
 'https://stackoverflow.com/questions/38318136/how-can-i-select-data-from-a-dask-dataframe-by-a-list-of-indices',
 'https://stackoverflow.com/questions/49480496/how-to-use-dask-pivot-table']

### Get post links for multiple pages

In [37]:
# serial code
page_posts_links = []
for page in range(1, 5):
    page_html = get_page_html_links(page_num=page)
    posts_links = get_post_links_per_page(page_html)
    
    page_posts_links.append(posts_links)

In [39]:
len(page_posts_links)

4

In [40]:
[len(l) for l in page_posts_links]

[50, 50, 50, 50]

**Parallel code: using `client.map()`**

We can get first the futures for every page html, and pass those futures as the iterator to get the links per page.

In [41]:
pages_html_futures = client.map(get_page_html_links, range(1,5))
wait(pages_html_futures) #wait until completed

DoneAndNotDoneFutures(done={<Future: finished, type: bs4.BeautifulSoup, key: get_page_html_links-11d659047375d225ac18ec43c616b9c6>, <Future: finished, type: bs4.BeautifulSoup, key: get_page_html_links-a7c3ef97cff5852f9cf430329620686f>, <Future: finished, type: bs4.BeautifulSoup, key: get_page_html_links-2aa8346c695e3d0dea84f03e0af0441e>, <Future: finished, type: bs4.BeautifulSoup, key: get_page_html_links-8a9b402c1b55ad011e624d2ef5a3707a>}, not_done=set())

**`wait()`**

Notice that here we used `wait()`, you can wait on a future or collection of futures using the `wait` function, which blocks until all futures are finished or have erred. This is useful when you need the all the futures to be completed to proceed with your computations. 

In [42]:
pages_html_futures[0]

<Future: finished, type: bs4.BeautifulSoup, key: get_page_html_links-a7c3ef97cff5852f9cf430329620686f>

### Exercise:

Using `client.map()` and the `pages_html_futures` you just got, to get the post's links for the four pages, in parallel.

In [44]:
#Solution
posts_links_futures = client.map(get_post_links_per_page, pages_html_futures)
posts_links_futures

[<Future: pending, key: get_post_links_per_page-e59b9076195aa7ce5173a2bd0b824c00>,
 <Future: pending, key: get_post_links_per_page-3e6f095a3188405dd3a83934aa3e2c2c>,
 <Future: pending, key: get_post_links_per_page-b80ec8b04812fbdfceeaa2595a15d2be>,
 <Future: pending, key: get_post_links_per_page-2647faa539ed27bcecdb0162388bc307>]

In [45]:
posts_links_futures[0]

<Future: finished, type: list, key: get_post_links_per_page-e59b9076195aa7ce5173a2bd0b824c00>

In [47]:
posts_links_futures[0].result()[:3]

['https://stackoverflow.com/questions/45545110/make-pandas-dataframe-apply-use-all-cores',
 'https://stackoverflow.com/questions/38882660/at-what-situation-i-can-use-dask-instead-of-apache-spark',
 'https://stackoverflow.com/questions/51361356/a-comparison-between-fastparquet-and-pyarrow']

**`as_completed()`**

In the example above we waited for the the `pages_html_futures` to finish before we proceed to get the `posts_links_futures`. However, we can get the `post_links_futures` for every page as the `pages_html_futures` finish. 

`as_completed()` return futures in the order in which they complete. It returns an iterator that yields the input future objects in the order in which they complete. 

In [49]:
from dask.distributed import as_completed

In [51]:
pages_html_futures = client.map(get_page_html_links, range(1,5), pure=False) #use pure=False to re-compute

post_links_futures = []
for p in as_completed(pages_html_futures):
    post_links_futures.append(client.submit(get_post_links_per_page, p))
    

In [52]:
post_links_futures

[<Future: finished, type: list, key: get_post_links_per_page-0cf7da3062e97088e778734c6fde9382>,
 <Future: finished, type: list, key: get_post_links_per_page-fa01a219db9bb609eef441177ddccdd8>,
 <Future: finished, type: list, key: get_post_links_per_page-d3653760d7632324fb0eb43ba260eea8>,
 <Future: finished, type: list, key: get_post_links_per_page-97849d9474dbb0f51f806a13402d3b20>]

## Grown-up example: Scrape, crawl and get SO data

Let's use all what we've learn in the examples above, to do something a bit more advanced. In this section, we graduate to a grownup example. You will learn how to parallelize a scrapping, crawling and get data workflow.

You'll be scraping multiple pages from https://stackoverflow.com/questions/, then you will be finding all the links for every post in each page and finally, getting some data from each post. You'll first see how the sequential code works, and then you'll use `futures` to do this in parallel

```
#NEEDS MOTIVATION, AND EXMAPLANATION OF WHTA WE ARE TRYING TO DO

#PROBABLY A PLOT SHOWING THE RESULTS (IMAGE)
```


Note about throttling:

When scrapping directly from the pages and not using the API, it is not clear what are the throttling limitations, but from experience we run into them pretty quickly.

The following examples, work as they are, if you change the number of pages you will likely hit a limit and be banned for few minutes. We will work around this towards the end, in the meantime avoid changing the number of pages

## Scrape, crawl, get data

## EDIT
We wrote some functions that get a page and clean the data from all the posts in that page and returns it as a list of dictionaries.


In [5]:
def request_html_page(url):
    """Given a url returns the html of that page
    """
    req = requests.get(url)
    html = bs(req.text, "html.parser")
    return html

In [6]:
def get_page_html_links(page_num, tag="dask", query_filter = "MostVotes"):
    """Given a SO questions page number returns a the html for that page number
    for a tag and query_filter.
    """
    base_url = "https://stackoverflow.com/questions/tagged/"
    
    page_url = f"{base_url}{tag}?sort={query_filter}&page={page_num}"

    page_html = request_html_page(page_url)
    
    return page_html

In [26]:
def get_post_links_per_page(html_page):
    """Given a SO questions html page, returns a list with the posts of that page.
    """
    question_href = html_page.find_all("a", class_="s-link")[2:-1]
    
    question_link = [f"https://stackoverflow.com{q['href']}" for q in question_href]
    
    return question_link

In [ ]:
def get_data(post_link):
    html_post = request_html_page(post_link)
    post_info = {}
    
    
    post_info["title"] = html_post.title.text
    post_info["question"] = html_post.find("div", class_="s-prose js-post-body").text
    
    answ = html_post.find("div", class_="answer") #this will gets us the first/most voted answer
    
    if answ:
        post_info["best_answer_votes"] = int(answ["data-score"])
    
        best_answer_author_obj = answ.find("span", itemprop="name")
        
        if best_answer_author_obj:
            best_answer_author = best_answer_author_obj.text
        else:
            best_answer_author = "comunity_post"

        post_info["best_answer_usrname"] = best_answer_author
    else:
        post_info["best_answer_votes"] = 0
        post_info["best_answer_usrname"] = "no-answer"

    
    return post_info

## Serial

In [ ]:
%%time

df_list =[]
for page_num in range(1, 3): #more than 2 pages and get trhottling issues
    page_html = get_page_html_links(page_num)
    posts_links = get_post_links_per_page(page_html)
    list_post_data = []
    
    for link in posts_links:
        p_data = get_data(link)
        list_post_data.append(p_data)

    df = pd.DataFrame(list_post_data)
    df_list.append(df)

In [ ]:
df_list[0].head()

## Parallel

### Get pages and links of posts

In [ ]:
%%time
pages_futures = client.map(get_page_html_links, range(1,3))
wait(pages_futures)

In [ ]:
%%time

posts_links_futures = client.map(get_post_links_per_page, pages_futures)
crawling = as_completed(posts_links_futures)

dfs_data = []
for future in crawling:
    list_links = future.result() # list of links per page
    df_data = []
    for link in list_links:
        fut_data = client.submit(get_data, link) 
        df_data.append(fut_data)

    dfs_data.append(df_data)
_ = wait(dfs_data)

## Avoid throttling using a cluster. 

In the example above, if we try to work with more pages, you will hit throttling issues. This is a problem, but we can solve it by scaling out to a lot of machines. When using a cluster each worker has its own public ip-address so it is like we are requesting from different machines. 

Let's create a coiled cluster and scrape a bigger number of pages:

In [ ]:
#Shutdown LocalCluster
client.shutdown()

In [ ]:
import coiled

In [ ]:
cluster = coiled.Cluster(n_workers=10, 
                        package_sync=True)

In [ ]:
client =  Client(cluster)
client

In [ ]:
%%time
pages_10_futures = client.map(get_page_html_links, range(1,11))
wait(pages_10_futures)

posts_links_futures = client.map(get_post_links_per_page, pages_10_futures)
crawling = as_completed(posts_links_futures)

dfs_data = []
for future in crawling:
    list_links = future.result() # list of links per page
    df_data = []
    for link in list_links:
        fut_data = client.submit(get_data, link) 
        df_data.append(fut_data)

    dfs_data.append(df_data)
_ = wait(dfs_data)

In [ ]:
len(dfs_data) #10 pages

At this point, we have the data to build each page dataframe. 

In [ ]:
dfs_data[0]

### Exercise 
Using futures, transform `dfs_data` into a list of pandas.Dataframe futures.

In [ ]:
## Solution
df_futures = client.map(pd.DataFrame, dfs_data)

In [ ]:
df_futures[0]

Now we have some pandas dataframes!!

### Exercise

Write a function that returns the `value_counts` of the `best_answer_usrname`, and apply it to the dataframe futures

In [ ]:
###Solution
def best_ans_val_counts(df):
    return df.best_answer_usrname.value_counts()

best_ans_val_counts_futures = client.map(best_ans_val_counts, df_futures)
best_ans_val_counts_futures[0].result()

### Exercise RE-WRITE NOT USE AS_COMPLETED, DO FIRST WITHOUT

Write a function that calculates the total amount of votes by best answer, apply it to the dataframe futures, and as they are completed accumulate the votes per user_name in a dictionary.

In [ ]:
#solution
def most_votes(df):
    return df.groupby("best_answer_usrname")['best_answer_votes'].sum()

most_votes_futures = client.map(most_votes, df_futures, pure=False)

In [ ]:
##solution
d = {}
for fut in as_completed(most_votes_futures):
    s = fut.result()
    fut_d = s.to_dict()
    for k, v in fut_d.items():
        if k in d:
            d[k] += v
        else:
            d[k] = v

In [ ]:
max(d, key=d.get)

In [ ]:
max(d.values())

### Dask dataframes API

Now we are on dataframe world, we can do pandas-like operations, for example.

We can do multiple operations on these dataframes using `futures` but at this point since we are working with dataframes we can use `dask.dataframes`. 

In [ ]:
import dask.dataframe as dd

In [ ]:
ddf_so = dd.from_delayed(df_futures)

In [ ]:
ddf_so

In [ ]:
ddf_so.columns

We can check which of the user tht got a best answer, has the most "best answers"

In [ ]:
ddf_so.best_answer_usrname.value_counts().compute()[:6]

In [ ]:
We can also check how many votes, this users got:

In [ ]:
ddf_so.groupby("best_answer_usrname")['best_answer_votes'].sum().compute()

## Extra:

Modify the following code, to get a different tag (e.g `tag="python"`) and re-run the experiment

```python
pages_futures = client.map(get_page_html_links, range(1,3))
wait(pages_futures)
```

Hint: You can use `client.map()` with `lambda` functions


In [ ]:
###Solution
pages_py_futures = client.map(lambda p: get_page_html_links(p, tag="python"), range(1, 5))
wait(pages_py_futures)

### Useful links

- https://tutorial.dask.org/05_futures.html

**Useful links**

* [Futures documentation](https://docs.dask.org/en/latest/futures.html)
* [Futures screencast](https://www.youtube.com/watch?v=07EiCpdhtDE)
* [Futures examples](https://examples.dask.org/futures.html)

### Next lesson

In the next lesson, you will get better at `dask.Dataframes`. We will re-cap the basics, but dive deeper into data formats (csv vs parquet),  learn about `pyarrow-strings`, shuffle operations, and other useful content that is not usually covered in the introductory material
